# Options on Futures

Everyone should be familiar with Black's model from FINM 33000.


## Formulas for Black's model

\$
P_{Black} = e^{-rT}[K N(-d_2) - F_0 N(-d_1)]
\$

\$
C_{Black} = e^{-rT}[F_0 N(d_1) - K N(d_2)]
\$

\$
d_1 = \frac{\ln(F_0/K) + 0.5\sigma^2 T}{\sigma\sqrt{T}}, \quad d_2 = d_1 - \sigma\sqrt{T}
\$



| Feature              | Equity Option               | Futures Option                   |
|----------------------|-----------------------------|----------------------------------|
| Underlying           | Stock/Index Spot ($S_t$)    | Futures Price ($F_t$)            |
| Carry cost           | $r - q$                     | Implicitly 0 (included in $F_t$) |
| Forward relationship | $F_0 = S_0 e^{(r - q) * T}$ | —                                |
| Pricing model        | Black–Scholes (1973)        | Black (1976)                     |
| Payoff               | $\max(S_T - K, 0)$          | $\max(F_T - K, 0)$               |
| Discounting          | $e^{-rT}$                   | $e^{-rT}$                        |

## Put-call parity in Black's model

\$
C_{Black} - P_{Black} = e^{-rT}(F_0 - K)
\$

\$
\Delta_C - \Delta_P = e^{-rT}
\$

## Common Misconceptions


### 1. Futures have no dividends, so it is never optimal to exercise calls early.

This conflates the theory of non-dividend paying stocks with futures, but the stock price represents a value today unlike the futures price.

$$
C_E(S_t, K) \ge S_t - e^{-r(T-t)} K \ge S_t - K
$$

Why doesn't this apply to futures?

### 2. Put-call parity applies to American options.

Put-call parity is still a useful approximation for American options with little early exercise value, but highly misleading for deep options.

### 3. By put-call parity, call delta minus put delta must be 100.

This is a multipart misconception:

a. Delta is the probability an option finishes in the money.

b. Applying put-call parity to American options.

c. Conflating $N(d_1)$ with call delta. That is the delta in a Black-Scholes model with no dividends.


## American exercise

Many options on futures have American exercise,
so we need to use models that support that.

The CME claims that most of its options are European here: https://www.cmegroup.com/education/courses/introduction-to-options/understanding-the-difference-european-vs-american-style-options.html

> The majority of CME Group options on futures are European style and can be exercised only at expiration.
 > Some of the notable exceptions that have American style expiration are the quarterly options on the
 > S&P500 futures contracts, SOFR options, and Treasury options.


That may well be the case if you consider all the options available to trade there,
but there are significant contingents that are American. The vast majority of options trading that I have supported has been American.

Popular American modeling choices:
* tree-based models with early exercise
* finite difference model
* Barone-Adesi/Whaley approximation
* Bjerksund/Stensland approximation

The key to using any of them is to use the relationship between the Black and Black-Scholes-Merton model.
That is, set the dividend rate `q` equal to the interest rate `r` to eliminate any drift of the underlying asset over the life of the option.

### Example American Pricing

You may already have code for pricing American options from your other courses. I will use QuantLib's implementation of
the Barone-Adesi-Whaley model here rather than write my own
to keep the focus on various issues when working with options market data, and not on the specifics of the pricing model.

## Implying Volatility From Real Market Data

Let's use this model to imply volatility from actual markets. First, we must
decide which price to use. Each strike (usually) has a bid and an offer, so a common
approach is to fit the mid-market price.

Buried in the code cells below are some issues and assumptions.
1. Interest rate is set to 4%. There are a few different approaches to setting this in practice
    * Use a discount rate model to set the rate.
    * Use market-implied rate (e.g., from a box spread) (European only)
    * Let the trader set the rate.
2. Actual/365 day count convention
    * Business/252 very common. Requires holiday management.
    * Other conventions are common as well.
    * Plain vanilla intraday time-to-expiration
3. American and European models are both used on the same options.
    * I am calculating this for some comparisons. Do not do both in practice.
    * Exception: de-americanization
        * Imply vol/model param from American prices
        * Calculate European prices with that parameter.
        * Continue modeling/analysis with European prices/vols.
4. Root-finding algorithms!!
    * More on this below.

In [1]:
import databento as db
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from finm37000 import (
    add_top_quantity,
    add_underlying,
    add_vol_plot,
    add_vol_range,
    add_volume_plot,
    add_width,
    aggregate_ohlcv,
    get_databento_api_key,
    get_options_chain,
    get_top_of_book,
    imply_american_vols,
    imply_european_vol,
    layout_vol,
    layout_volume,
    make_top_subplots,
    temp_env,
    tz_chicago,
)

with temp_env(DATABENTO_API_KEY=get_databento_api_key()):
    client = db.Historical()

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


### Option Expiration Cycle

The first example uses the month crude oil options. These are American options with underlying $CL$ futures.
There are a variety of options on $CL$ futures: monthly, weekly, and options on spreads.
The monthly contracts expire 3 days before the corresponding future.

The CME provides lots of information about their products, and you should consult their website
when you need to learn about them. For example, here is the monthly crude oil contract we are going
to examine: https://www.cmegroup.com/markets/energy/crude-oil/light-sweet-crude.volume.options.html#optionProductId=190

Look at the breadth of available options beyond this. Some are actively traded, some are barely traded.

In [2]:
parent_option = "LO"
underlying_symbol = "CLX5"
option_label = "LOX5"
start = pd.Timestamp("2025-10-10T12:00:00", tz=tz_chicago)
end = pd.Timestamp("2025-10-10T12:01:00", tz=tz_chicago)
options_chain = get_options_chain(
    parent=parent_option,
    underlying=underlying_symbol,
    start=start,
    client=client,
)
options_chain

,ts_event,rtype,publisher_id,instrument_id,raw_symbol,security_update_action,instrument_class,min_price_increment,display_factor,expiration,...,underlying_product,maturity_month,maturity_day,maturity_week,user_defined_instrument,contract_multiplier_unit,flow_schedule_type,tick_rule,symbol,years_to_expiration
ts_recv,,,,,,,,,,,,,,,,,,,,,
2025-10-10 00:00:00+00:00,2025-10-05 11:03:23.965000+00:00,19,1,42244439,LOX5 P250,A,P,0.01,0.01,2025-10-16 18:30:00+00:00,...,16,11,255,255,N,127,127,255,LOX5 P250,0.01661
2025-10-10 00:00:00+00:00,2025-10-05 11:02:49.415000+00:00,19,1,42243447,LOX5 C250,A,C,0.01,0.01,2025-10-16 18:30:00+00:00,...,16,11,255,255,N,127,127,255,LOX5 C250,0.01661
2025-10-10 00:00:00+00:00,2025-10-05 11:03:29.065000+00:00,19,1,42244440,LOX5 P500,A,P,0.01,0.01,2025-10-16 18:30:00+00:00,...,16,11,255,255,N,127,127,255,LOX5 P500,0.01661
2025-10-10 00:00:00+00:00,2025-10-05 11:03:03.415000+00:00,19,1,42243111,LOX5 C500,A,C,0.01,0.01,2025-10-16 18:30:00+00:00,...,16,11,255,255,N,127,127,255,LOX5 C500,0.01661
2025-10-10 00:00:00+00:00,2025-10-05 11:02:32.765000+00:00,19,1,42117702,LOX5 C600,A,C,0.01,0.01,2025-10-16 18:30:00+00:00,...,16,11,255,255,N,127,127,255,LOX5 C600,0.01661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-10 00:00:00+00:00,2025-10-05 11:02:48.365000+00:00,19,1,42244736,LOX5 C21500,A,C,0.01,0.01,2025-10-16 18:30:00+00:00,...,16,11,255,255,N,127,127,255,LOX5 C21500,0.01661
2025-10-10 00:00:00+00:00,2025-10-05 11:03:10.015000+00:00,19,1,42244438,LOX5 C22000,A,C,0.01,0.01,2025-10-16 18:30:00+00:00,...,16,11,255,255,N,127,127,255,LOX5 C22000,0.01661
2025-10-10 00:00:00+00:00,2025-10-05 11:03:14.565000+00:00,19,1,42243142,LOX5 P22000,A,P,0.01,0.01,2025-10-16 18:30:00+00:00,...,16,11,255,255,N,127,127,255,LOX5 P22000,0.01661


In [3]:
top_prices = get_top_of_book(
    symbols=[underlying_symbol, *options_chain["raw_symbol"]],
    start=start,
    end=end,
    client=client,
)

In [4]:
underlying_price = top_prices["midprice"].loc[underlying_symbol]
with_top_prices = options_chain.join(top_prices, on="symbol").dropna(subset="midprice")
with_top_prices["underlying_price"] = underlying_price
with_top_prices["interest_rate"] = 0.04
with_vols = with_top_prices.assign(
    **imply_american_vols(
        with_top_prices, futures_price=underlying_price, risk_free_rate=0.04
    )
)
with_vols["european_vol"] = with_vols.apply(
    imply_european_vol,
    axis=1,
)

# Separate options into calls and puts
call_options = with_vols[with_vols["instrument_class"] == "C"]
put_options = with_vols[with_vols["instrument_class"] == "P"]

Cannot find OptionType.PUT vol between lb=1e-05 and ub=4 at strike 220.0: lower_vol=160.95802660126256 target=161.065 upper_vol=161.054296997451
  F=58.935 T=0.01660958904109589 r=0.04 mid=161.065
Cannot find OptionType.PUT vol between lb=1e-05 and ub=4 at strike 225.0: lower_vol=165.9547057867238 target=166.065 upper_vol=166.0395102025281
  F=58.935 T=0.01660958904109589 r=0.04 mid=166.065


### Simplest implied vol skew

Most presentations of skew avoid showing the reality of the bids and offers on calls and puts in a real options market.

For example, a common choice is to only plot strikes near the underlying price and use the mid-price to imply the vol.

In [5]:
near_atm = (40.0, 70.0)
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=call_options,
    name="Midprice vol",
    y_col="iv_midprice",
    strike_range=near_atm,
)
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=option_label, detail=end)
fig.show()

That looks reasonable with maybe some weird jumps at some points. But there is more in the market than that!

### Puts and Calls
For example, we can look at both puts and calls:

In [6]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=put_options,
    name="Put midprice vol",
    y_col="iv_midprice",
    strike_range=near_atm,
)
add_vol_plot(
    fig=fig,
    vol_df=call_options,
    name="Call midprice vol",
    y_col="iv_midprice",
    strike_range=near_atm,
)
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig, label=option_label, detail=end)
fig.show()

### Mixing targets and models

In [7]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=put_options,
    name="Put midprice vol",
    y_col="iv_midprice",
    strike_range=near_atm,
)
add_vol_plot(
    fig=fig,
    vol_df=put_options,
    name="Put ask vol",
    y_col="iv_ask",
    strike_range=near_atm,
)
add_vol_plot(
    fig=fig,
    vol_df=put_options,
    name="Put european midprice vol",
    y_col="european_vol",
    strike_range=near_atm,
)
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=option_label, detail=end)
fig.show()

### All strikes

Now let's zoom out to see all the strikes.

In [8]:
fig = go.Figure()
add_vol_plot(fig=fig, vol_df=put_options, name="Put midprice vol", y_col="iv_midprice")
add_vol_plot(
    fig=fig, vol_df=call_options, name="Call midprice vol", y_col="iv_midprice"
)
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=option_label, detail=end)
fig.show()

Yikes! It really struggles to get implied vols at the market midprice for those deep puts!

POLL: Are those puts deep in or deep out of the money?


We can also compare to the Black vol, i.e., as if it is European.


In [9]:
fig = go.Figure()
add_vol_plot(fig=fig, vol_df=put_options, name="Put midprice vol", y_col="iv_midprice")
add_vol_plot(fig=fig, vol_df=put_options, name="Put ask vol", y_col="iv_ask")
add_vol_plot(
    fig=fig, vol_df=put_options, name="Put european midprice vol", y_col="european_vol"
)
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=option_label, detail=end)
fig.show()

### Bid/ask range of vol: Calls

We can look at the range of vols implied by the bid/ask:

In [10]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=call_options,
    name="Call midprice vol",
    y_col="iv_midprice",
    strike_range=near_atm,
)
add_vol_range(fig, vol_df=call_options, strike_range=near_atm)
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig, label=option_label, detail=end)
fig.show()

### Bid/ask range of vols: Puts

In [11]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=put_options,
    name="Put midprice vol",
    y_col="iv_midprice",
    strike_range=near_atm,
)
add_vol_range(fig, vol_df=put_options, strike_range=near_atm)
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=option_label, detail=end)
fig.show()

Why do the bars disappear at some points?

In [12]:
price_vol_cols = [
    "symbol",
    "strike_price",
    "bidq",
    "bid",
    "ask",
    "askq",
    "iv_bid",
    "iv_ask",
]

In [13]:
strikes = [45, 60, 70]
strike_mat = np.broadcast_to(
    put_options["strike_price"].to_numpy().reshape(-1, 1),
    (len(put_options), len(strikes)),
)
strike_mask = np.isclose(strike_mat, strikes).any(axis=1)
put_options[strike_mask][price_vol_cols]

,symbol,strike_price,bidq,bid,ask,askq,iv_bid,iv_ask
ts_recv,,,,,,,,
2025-10-10 00:00:00+00:00,LOX5 P4500,45.0,0.0,NaN,0.02,1022.0,NaN,0.902719
2025-10-10 00:00:00+00:00,LOX5 P6000,60.0,8.0,1.60,1.64,1.0,0.321193,0.335598
2025-10-10 00:00:00+00:00,LOX5 P7000,70.0,1.0,11.04,11.12,2.0,NaN,0.692894


Look at the range of vols on some of those bars! For example, the 65 strike:

In [14]:
put_options[np.isclose(put_options["strike_price"], 65)][price_vol_cols]

,symbol,strike_price,bidq,bid,ask,askq,iv_bid,iv_ask
ts_recv,,,,,,,,
2025-10-10 00:00:00+00:00,LOX5 P6500,65.0,2.0,6.07,6.14,1.0,0.313475,0.463387


In [15]:
with_vols[np.isclose(with_vols["strike_price"], 65)][price_vol_cols]

,symbol,strike_price,bidq,bid,ask,askq,iv_bid,iv_ask
ts_recv,,,,,,,,
2025-10-10 00:00:00+00:00,LOX5 P6500,65.0,2.0,6.07,6.14,1.0,0.313475,0.463387
2025-10-10 00:00:00+00:00,LOX5 C6500,65.0,16.0,0.04,0.05,703.0,0.410815,0.427467


#### Quiz/Interview Question

If 65 put price is \\$6.10 and the call price is \\$0.05, what is the futures price?


In [16]:
t = with_vols["years_to_expiration"].iloc[0]
r = with_vols["interest_rate"].iloc[0]
-6.05 * np.exp(r * t) + 65, underlying_price

(np.float64(58.94597914390461), np.float64(58.935))

### With trade volume
Do all the discrepancies above matter? Which strikes are actually trading?

In [17]:
volume_start = pd.Timestamp("2025-10-10T07:00:00", tz=tz_chicago)
volume_end = pd.Timestamp("2025-10-10T16:00:00", tz=tz_chicago)
trades = client.timeseries.get_range(
    dataset=db.Dataset.GLBX_MDP3,
    symbols=[underlying_symbol, *options_chain["raw_symbol"]],
    start=volume_start,
    end=volume_end,
    schema="trades",
).to_df()
trade_volume = aggregate_ohlcv(trades)

In [18]:
with_volume = with_vols.merge(trade_volume, on="symbol")
call_volume = with_volume[with_volume["instrument_class"] == "C"]
put_volume = with_volume[with_volume["instrument_class"] == "P"]
underlying_ohlcv = trade_volume.loc[underlying_symbol]
underlying_low, underlying_high = underlying_ohlcv["low"], underlying_ohlcv["high"]

In [19]:
fig = go.Figure()
add_volume_plot(fig=fig, volume_df=call_volume, name="Call volume")
add_volume_plot(fig=fig, volume_df=put_volume, name="Put volume")
add_underlying(
    fig=fig, underlying_price=underlying_low, text="low", position="top left"
)
add_underlying(fig=fig, underlying_price=underlying_high, text="high")
layout_volume(
    fig=fig, label=option_label, detail=f"Between {volume_start} and {volume_end}"
)
fig.show()

In [20]:
fig = make_top_subplots(option_label)
add_top_quantity(fig, call_volume, "Call")
add_top_quantity(fig, put_volume, "Put")
add_width(fig, call_volume, "Call")
add_width(fig, put_volume, "Put")
fig.update_layout(
    template="plotly_white",
)
fig.show()

### OTM only

Extremely common approach is to only use OTM options to imply vols.

In [21]:
otm_options = pd.concat(
    [
        call_options[call_options["strike_price"] >= underlying_price],
        put_options[put_options["strike_price"] <= underlying_price],
    ]
).sort_values(by="strike_price")

In [22]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=otm_options,
    name="OTM midprice vol",
    y_col="iv_midprice",
    strike_range=near_atm,
)
add_vol_range(fig, vol_df=otm_options, strike_range=near_atm)
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=option_label, detail=end)
fig.show()

In [23]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=otm_options,
    name="OTM bid vol",
    y_col="iv_bid",
)
add_vol_plot(
    fig=fig,
    vol_df=otm_options,
    name="OTM ask vol",
    y_col="iv_ask",
)
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=option_label, detail=end)
fig.show()

## Other Markets

That was a close look at one expiration in crude oil, but there is a wide range of other options on futures.
You can see the top trading ones at the CME: https://www.cmegroup.com/markets/options.html#market-activity

A good way to find the symbols you need

* Go to the CME website and look for the futures underlying
* Navigate to Volume and OI or Specs
* Select Options.
* Look at the volume and open interest if you are not 100% sure which options
series you are looking for. You typically are looking for the most traded one.
* Select Contract Specs to figure out the symbols.



### Corn Options

https://www.cmegroup.com/markets/agriculture/grains/corn.html

`OZC` is the symbol for monthly options on futures.

In [24]:
parent_option = "OZC"
underlying_symbol = "ZCZ5"
option_label = "OZCZ5"
options_chain = get_options_chain(
    parent=parent_option,
    underlying=underlying_symbol,
    start=start,
    client=client,
)
top_prices = get_top_of_book(
    symbols=[underlying_symbol, *options_chain["raw_symbol"]],
    start=start,
    end=end,
    client=client,
)
trades = client.timeseries.get_range(
    dataset=db.Dataset.GLBX_MDP3,
    symbols=[underlying_symbol, *options_chain["raw_symbol"]],
    start=volume_start,
    end=volume_end,
    schema="trades",
).to_df()
trade_volume = aggregate_ohlcv(trades)

In [25]:
underlying_price = top_prices["midprice"].loc[underlying_symbol]
with_top_prices = options_chain.join(top_prices, on="symbol").dropna(subset="midprice")
with_top_prices["underlying_price"] = underlying_price
with_top_prices["interest_rate"] = 0.04
with_vols = with_top_prices.assign(
    **imply_american_vols(
        with_top_prices, futures_price=underlying_price, risk_free_rate=0.04
    )
)
with_vols["european_vol"] = with_vols.apply(
    imply_european_vol,
    axis=1,
)

with_volume = with_vols.merge(trade_volume, on="symbol")
underlying_ohlcv = trade_volume.loc[underlying_symbol]
underlying_low, underlying_high = underlying_ohlcv["low"], underlying_ohlcv["high"]

In [26]:
call_volume = with_volume[with_volume["instrument_class"] == "C"]
put_volume = with_volume[with_volume["instrument_class"] == "P"]
otm_options = pd.concat(
    [
        call_volume[call_volume["strike_price"] >= underlying_price],
        put_volume[put_volume["strike_price"] <= underlying_price],
    ]
).sort_values(by="strike_price")

In [27]:
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=otm_options,
    name="OTM midprice vol",
    y_col="iv_midprice",
)
add_vol_range(fig, vol_df=otm_options)
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=option_label, detail=end)
fig.show()

Uh-oh! What is going on here? Look at the data! Do you see what has gone wrong?

In [28]:
otm_options[price_vol_cols].sort_values(by="strike_price")

,symbol,strike_price,bidq,bid,ask,askq,iv_bid,iv_ask
0,OZCZ5 P0335,335.0,6662.0,0.020,0.125,295.0,0.218684,0.268262
1,OZCZ5 P0360,360.0,711.0,0.375,0.500,68585.0,0.220183,0.232065
3,OZCZ5 P0365,365.0,110.0,0.500,0.625,67778.0,0.212699,0.222350
2,OZCX5 P0365,365.0,4916.0,0.020,0.125,52126.0,0.234330,0.292557
4,OZCZ5 P0370,370.0,68124.0,0.500,0.625,50.0,0.193348,0.202369
...,...,...,...,...,...,...,...,...
85,OZCZ5 C0470,470.0,59818.0,0.375,0.500,55403.0,0.205416,0.216297
86,OZCZ5 C0475,475.0,110.0,0.375,0.500,68354.0,0.219016,0.230390
87,OZCZ5 C0480,480.0,68133.0,0.250,0.375,266.0,0.217713,0.232330
88,OZCZ5 C0500,500.0,58985.0,0.125,0.375,58962.0,0.242221,0.283063


Look at the corn option calendar specification: https://www.cmegroup.com/markets/agriculture/grains/corn.contractSpecs.options.html#optionProductId=301

There will be three consecutive month contracts for near term months, but there are only futures for H, K, N, U, and Z.

So when we asked for all options on `OZC` with underlying `ZCZ5`, that includes October (expired) and November options that expire into
December futures.


In [29]:
option_expirations = ("OZCX5", "OZCZ5")
call_volume = {}
put_volume = {}
otm_options = {}
for option_expiration in option_expirations:
    calls = with_volume[with_volume["instrument_class"] == "C"]
    puts = with_volume[with_volume["instrument_class"] == "P"]
    calls = calls[calls["symbol"].str.startswith(option_expiration)]
    puts = puts[puts["symbol"].str.startswith(option_expiration)]
    call_volume[option_expiration] = calls
    put_volume[option_expiration] = puts
    otm_options[option_expiration] = pd.concat(
        [
            calls[calls["strike_price"] >= underlying_price],
            puts[puts["strike_price"] <= underlying_price],
        ]
    ).sort_values(by="strike_price")

In [30]:
fig = go.Figure()
for option_expiration in option_expirations:
    add_vol_plot(
        fig=fig,
        vol_df=otm_options[option_expiration],
        name=f"OTM {option_expiration} bid vol",
        y_col="iv_bid",
    )
    add_vol_plot(
        fig=fig,
        vol_df=otm_options[option_expiration],
        name=f"OTM {option_expiration} ask vol",
        y_col="iv_ask",
    )
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=underlying_symbol, detail=end)
fig.show()

In [31]:
fig = go.Figure()
for option_expiration in option_expirations:
    add_vol_plot(
        fig=fig,
        vol_df=otm_options[option_expiration],
        name=f"OTM {option_expiration} midprice vol",
        y_col="iv_midprice",
    )
    add_vol_range(fig, vol_df=otm_options[option_expiration])
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=option_label, detail=end)
fig.show()

#### Weighted midprices

A common approach to smoothing out wide markets is to use the top-of-book quantity to weight the mid-prices.

That is definitely useful information to include, but naive use does not necessarily help very much:

In [32]:
fig = go.Figure()
option_expiration = "OZCX5"
add_vol_plot(
    fig=fig,
    vol_df=otm_options[option_expiration],
    name=f"OTM {option_expiration} midprice vol",
    y_col="iv_midprice",
)
add_vol_plot(
    fig=fig,
    vol_df=otm_options[option_expiration],
    name=f"OTM {option_expiration} weighted midprice vol",
    y_col="iv_weighted_midprice",
)
add_vol_range(fig, vol_df=otm_options[option_expiration])
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=option_expiration, detail=end)
fig.show()

Notice how the weighted midprice can be jerked to either side of the market.

In [33]:
df = otm_options[option_expiration]
df[np.isclose(df["strike_price"], 370)][price_vol_cols]

,symbol,strike_price,bidq,bid,ask,askq,iv_bid,iv_ask
5,OZCX5 P0370,370.0,9067.0,0.02,0.125,478.0,0.210867,0.264426


POLL:

How would you weight an option market like this?

Let's take round numbers:
|bid quantity | bid | ask | ask quantity |
|-------------|-----|-----|--------------|
|9_500 | 0.02 | 0.10 | 500 |

#### The top of the option order books

Take a moment to zoom in on the visually missing quantities near the ATM.

In [34]:
fig = make_top_subplots(option_expiration)
add_top_quantity(fig, otm_options[option_expiration], "OTM")
add_width(fig, otm_options[option_expiration], "OTM")
fig.update_layout(
    template="plotly_white",
)
fig.show()

In [35]:
option_expiration = "OZCZ5"
fig = go.Figure()
add_vol_plot(
    fig=fig,
    vol_df=otm_options[option_expiration],
    name=f"OTM {option_expiration} midprice vol",
    y_col="iv_midprice",
)
add_vol_plot(
    fig=fig,
    vol_df=otm_options[option_expiration],
    name=f"OTM {option_expiration} weighted midprice vol",
    y_col="iv_weighted_midprice",
)
add_vol_range(fig, vol_df=otm_options[option_expiration])
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=option_expiration, detail=end)
fig.show()

In [36]:
fig = make_top_subplots(option_expiration)
add_top_quantity(fig, otm_options[option_expiration], "OTM")
add_width(fig, otm_options[option_expiration], "OTM")
fig.update_layout(
    template="plotly_white",
)
fig.show()

In [37]:
fig = go.Figure()
for option_expiration in option_expirations:
    add_volume_plot(
        fig=fig,
        volume_df=call_volume[option_expiration],
        name=f"{option_expiration} Call volume",
    )
    add_volume_plot(
        fig=fig,
        volume_df=put_volume[option_expiration],
        name=f"{option_expiration} Put volume",
    )

add_underlying(
    fig=fig, underlying_price=underlying_low, text="low", position="top left"
)
add_underlying(fig=fig, underlying_price=underlying_high, text="high")
layout_volume(
    fig=fig, label=underlying_symbol, detail=f"Between {volume_start} and {volume_end}"
)
fig.show()

How about looking a little further out on expiration? Here are options trading against March corn.

In [38]:
parent_option = "OZC"
underlying_symbol = "ZCH6"
option_expirations = (
    "OZCF6",
    "OZCG6",
    "OZCH6",
)  # On October 10, February is not trading yet
options_chain = get_options_chain(
    parent=parent_option,
    underlying=underlying_symbol,
    start=start,
    client=client,
)
top_prices = get_top_of_book(
    symbols=[underlying_symbol, *options_chain["raw_symbol"]],
    start=start,
    end=end,
    client=client,
)
trades = client.timeseries.get_range(
    dataset=db.Dataset.GLBX_MDP3,
    symbols=[underlying_symbol, *options_chain["raw_symbol"]],
    start=volume_start,
    end=volume_end,
    schema="trades",
).to_df()
trade_volume = aggregate_ohlcv(trades)

In [39]:
underlying_price = top_prices["midprice"].loc[underlying_symbol]
with_top_prices = options_chain.join(top_prices, on="symbol").dropna(subset="midprice")
with_top_prices["underlying_price"] = underlying_price
with_top_prices["interest_rate"] = 0.04
with_vols = with_top_prices.assign(
    **imply_american_vols(
        with_top_prices, futures_price=underlying_price, risk_free_rate=0.04
    )
)
with_vols["european_vol"] = with_vols.apply(
    imply_european_vol,
    axis=1,
)

with_volume = with_vols.merge(trade_volume, on="symbol")
underlying_ohlcv = trade_volume.loc[underlying_symbol]
underlying_low, underlying_high = underlying_ohlcv["low"], underlying_ohlcv["high"]
call_volume = {}
put_volume = {}
otm_options = {}
for option_expiration in option_expirations:
    calls = with_volume[with_volume["instrument_class"] == "C"]
    puts = with_volume[with_volume["instrument_class"] == "P"]
    calls = calls[calls["symbol"].str.startswith(option_expiration)]
    puts = puts[puts["symbol"].str.startswith(option_expiration)]
    call_volume[option_expiration] = calls
    put_volume[option_expiration] = puts
    otm_options[option_expiration] = pd.concat(
        [
            calls[calls["strike_price"] >= underlying_price],
            puts[puts["strike_price"] <= underlying_price],
        ]
    ).sort_values(by="strike_price")

In [40]:
fig = go.Figure()
for option_expiration in option_expirations:
    add_vol_plot(
        fig=fig,
        vol_df=otm_options[option_expiration],
        name=f"OTM {option_expiration} bid vol",
        y_col="iv_bid",
    )
    add_vol_plot(
        fig=fig,
        vol_df=otm_options[option_expiration],
        name=f"OTM {option_expiration} ask vol",
        y_col="iv_ask",
    )
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=underlying_symbol, detail=end)
fig.show()

In [41]:
fig = go.Figure()
for option_expiration in option_expirations:
    add_vol_plot(
        fig=fig,
        vol_df=otm_options[option_expiration],
        name=f"OTM {option_expiration} midprice vol",
        y_col="iv_midprice",
    )
    add_vol_range(fig, vol_df=otm_options[option_expiration])
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=option_label, detail=end)
fig.show()

In [42]:
fig = go.Figure()
for option_expiration in option_expirations:
    add_volume_plot(
        fig=fig,
        volume_df=call_volume[option_expiration],
        name=f"{option_expiration} Call volume",
    )
    add_volume_plot(
        fig=fig,
        volume_df=put_volume[option_expiration],
        name=f"{option_expiration} Put volume",
    )

add_underlying(
    fig=fig, underlying_price=underlying_low, text="low", position="top left"
)
add_underlying(fig=fig, underlying_price=underlying_high, text="high")
layout_volume(
    fig=fig, label=underlying_symbol, detail=f"Between {volume_start} and {volume_end}"
)
# fig.update_layout(bargap=0.1)
fig.show()

#### May Options

Going out even further, the trading activity drops off.

In [43]:
parent_option = "OZC"
underlying_symbol = "ZCN6"
option_expirations = ("OZCN6",)
options_chain = get_options_chain(
    parent=parent_option,
    underlying=underlying_symbol,
    start=start,
    client=client,
)
top_prices = get_top_of_book(
    symbols=[underlying_symbol, *options_chain["raw_symbol"]],
    start=start,
    end=end,
    client=client,
)
trades = client.timeseries.get_range(
    dataset=db.Dataset.GLBX_MDP3,
    symbols=[underlying_symbol, *options_chain["raw_symbol"]],
    start=volume_start,
    end=volume_end,
    schema="trades",
).to_df()
trade_volume = aggregate_ohlcv(trades)

In [44]:
underlying_price = top_prices["midprice"].loc[underlying_symbol]
with_top_prices = options_chain.join(top_prices, on="symbol").dropna(subset="midprice")
with_top_prices["underlying_price"] = underlying_price
with_top_prices["interest_rate"] = 0.04
with_vols = with_top_prices.assign(
    **imply_american_vols(
        with_top_prices, futures_price=underlying_price, risk_free_rate=0.04
    )
)
with_vols["european_vol"] = with_vols.apply(
    imply_european_vol,
    axis=1,
)

with_volume = with_vols.merge(trade_volume, on="symbol")
underlying_ohlcv = trade_volume.loc[underlying_symbol]
underlying_low, underlying_high = underlying_ohlcv["low"], underlying_ohlcv["high"]

In [45]:
call_volume = {}
put_volume = {}
otm_options = {}
for option_expiration in option_expirations:
    calls = with_volume[with_volume["instrument_class"] == "C"]
    puts = with_volume[with_volume["instrument_class"] == "P"]
    calls = calls[calls["symbol"].str.startswith(option_expiration)]
    puts = puts[puts["symbol"].str.startswith(option_expiration)]
    call_volume[option_expiration] = calls
    put_volume[option_expiration] = puts
    otm_options[option_expiration] = pd.concat(
        [
            calls[calls["strike_price"] >= underlying_price],
            puts[puts["strike_price"] <= underlying_price],
        ]
    ).sort_values(by="strike_price")

In [46]:
fig = go.Figure()
for option_expiration in option_expirations:
    add_vol_plot(
        fig=fig,
        vol_df=otm_options[option_expiration],
        name=f"OTM {option_expiration} midprice vol",
        y_col="iv_midprice",
    )
    add_vol_range(fig, vol_df=otm_options[option_expiration])
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=option_label, detail=end)
fig.show()

In [47]:
fig = go.Figure()
for option_expiration in option_expirations:
    add_volume_plot(
        fig=fig,
        volume_df=call_volume[option_expiration],
        name=f"{option_expiration} Call volume",
    )
    add_volume_plot(
        fig=fig,
        volume_df=put_volume[option_expiration],
        name=f"{option_expiration} Put volume",
    )

add_underlying(
    fig=fig, underlying_price=underlying_low, text="low", position="top left"
)
add_underlying(fig=fig, underlying_price=underlying_high, text="high")
layout_volume(
    fig=fig, label=underlying_symbol, detail=f"Between {volume_start} and {volume_end}"
)
# fig.update_layout(bargap=0.1)
fig.show()

### Gold Options

Let's be more judicious with how we pick our expirations this time.

Look at the expirations, open interest, and volume for gold options at the CME: https://www.cmegroup.com/markets/metals/precious/gold.volume.options.html#optionProductId=192


Both `OGX5` and `OGZ5` expire into the same future, so there are twice as many strikes for that same
underlying, and Databento requires us to break our queries into reasonable sizes.

In [48]:
parent_option = "OG"
option_expirations = (
    "OGX5",
    "OGZ5",
)
underlying_symbol = "GCZ5"
options_chain = get_options_chain(
    parent=parent_option,
    underlying=underlying_symbol,
    start=start,
    client=client,
)

In [49]:
top_prices = {}
trades = {}
trade_volume = {}
for option_expiration in option_expirations:
    expiration_chain = options_chain[
        options_chain["raw_symbol"].str.startswith(option_expiration)
    ]
    top_prices[option_expiration] = get_top_of_book(
        symbols=[underlying_symbol, *expiration_chain["raw_symbol"]],
        start=start,
        end=end,
        client=client,
    )
    trades[option_expiration] = client.timeseries.get_range(
        dataset=db.Dataset.GLBX_MDP3,
        symbols=[underlying_symbol, *expiration_chain["raw_symbol"]],
        start=volume_start,
        end=volume_end,
        schema="trades",
    ).to_df()
    trade_volume[option_expiration] = aggregate_ohlcv(trades[option_expiration])

In [50]:
with_volume = {}
for option_expiration in option_expirations:
    underlying_price = top_prices[option_expiration]["midprice"].loc[underlying_symbol]
    with_top_prices = options_chain.join(
        top_prices[option_expiration], on="symbol"
    ).dropna(subset="midprice")
    with_top_prices["underlying_price"] = underlying_price
    with_top_prices["interest_rate"] = 0.04
    with_vols = with_top_prices.assign(
        **imply_american_vols(
            with_top_prices, futures_price=underlying_price, risk_free_rate=0.04
        )
    )
    with_vols["european_vol"] = with_vols.apply(
        imply_european_vol,
        axis=1,
    )

    with_volume[option_expiration] = with_vols.merge(
        trade_volume[option_expiration], on="symbol"
    )
    underlying_ohlcv = trade_volume[option_expiration].loc[underlying_symbol]
    underlying_low, underlying_high = underlying_ohlcv["low"], underlying_ohlcv["high"]

Cannot find OptionType.CALL vol between lb=1e-05 and ub=4 at strike 3440.0: lower_vol=554.9529483584641 target=554.35 upper_vol=1568.0087312504324
  F=3996.05 T=0.04937214611872146 r=0.04 mid=554.35
Cannot find OptionType.CALL vol between lb=1e-05 and ub=4 at strike 3445.0: lower_vol=549.9628130436681 target=549.9000000000001 upper_vol=1566.057431575895
  F=3996.05 T=0.04937214611872146 r=0.04 mid=549.9000000000001
Cannot find OptionType.CALL vol between lb=1e-05 and ub=4 at strike 3455.0: lower_vol=539.9825424140762 target=539.9000000000001 upper_vol=1562.1642107084126
  F=3996.05 T=0.04937214611872146 r=0.04 mid=539.9000000000001
Cannot find OptionType.CALL vol between lb=1e-05 and ub=4 at strike 3465.0: lower_vol=530.0022717844842 target=529.95 upper_vol=1558.2834548641788
  F=3996.05 T=0.04937214611872146 r=0.04 mid=529.95
Cannot find OptionType.CALL vol between lb=1e-05 and ub=4 at strike 3510.0: lower_vol=485.0910539513201 target=485.0 upper_vol=1540.9730422035757
  F=3996.05 T=0

In [51]:
call_volume = {}
put_volume = {}
otm_options = {}
for option_expiration in option_expirations:
    df = with_volume[option_expiration]
    calls = df[df["instrument_class"] == "C"]
    puts = df[df["instrument_class"] == "P"]
    calls = calls[calls["symbol"].str.startswith(option_expiration)]
    puts = puts[puts["symbol"].str.startswith(option_expiration)]
    call_volume[option_expiration] = calls
    put_volume[option_expiration] = puts
    otm_options[option_expiration] = pd.concat(
        [
            calls[calls["strike_price"] >= underlying_price],
            puts[puts["strike_price"] <= underlying_price],
        ]
    ).sort_values(by="strike_price")

In [52]:
fig = go.Figure()
for option_expiration in option_expirations:
    add_vol_plot(
        fig=fig,
        vol_df=otm_options[option_expiration],
        name=f"OTM {option_expiration} bid vol",
        y_col="iv_bid",
    )
    add_vol_plot(
        fig=fig,
        vol_df=otm_options[option_expiration],
        name=f"OTM {option_expiration} ask vol",
        y_col="iv_ask",
    )
add_underlying(fig=fig, underlying_price=underlying_price)
layout_vol(fig=fig, label=underlying_symbol, detail=end)
fig.show()

In [53]:
fig = go.Figure()
for option_expiration in option_expirations:
    add_volume_plot(
        fig=fig,
        volume_df=call_volume[option_expiration],
        name=f"{option_expiration} Call volume",
    )
    add_volume_plot(
        fig=fig,
        volume_df=put_volume[option_expiration],
        name=f"{option_expiration} Put volume",
    )

add_underlying(
    fig=fig, underlying_price=underlying_low, text="low", position="top left"
)
add_underlying(fig=fig, underlying_price=underlying_high, text="high")
layout_volume(
    fig=fig, label=underlying_symbol, detail=f"Between {volume_start} and {volume_end}"
)
fig.show()

In [54]:
near_atm = (3700, 5000)
fig = go.Figure()
for option_expiration in option_expirations:
    call_df = call_volume[option_expiration]
    call_df = call_df[
        (call_df["strike_price"] >= near_atm[0])
        & (call_df["strike_price"] <= near_atm[1])
    ]
    put_df = put_volume[option_expiration]
    put_df = put_df[
        (put_df["strike_price"] >= near_atm[0])
        & (put_df["strike_price"] <= near_atm[1])
    ]
    add_volume_plot(fig=fig, volume_df=call_df, name=f"{option_expiration} Call volume")
    add_volume_plot(fig=fig, volume_df=put_df, name=f"{option_expiration} Put volume")

add_underlying(
    fig=fig, underlying_price=underlying_low, text="low", position="top left"
)
add_underlying(fig=fig, underlying_price=underlying_high, text="high")
layout_volume(
    fig=fig, label=underlying_symbol, detail=f"Between {volume_start} and {volume_end}"
)
fig.show()

### Additional Considerations

Each market will have its own pattern of expirations and trading activity that determine where opportunities are available.

A few bear special mention that won't be pursued in detail here but are worth knowing if your career takes you in this direction.

#### SOFR Options

`SR3` are some of the most traded options on the CME.
Their pricing requires extra care due to the way they are priced.

Prices are quotes as $100(1-r)$ or $1000(1-r)$ where $r$ is a forward interest rate (annualized with Actual/360 conventions).

Example:
* `SR3H6` price of 96.395 (or 9639.5) corresponds to an interest rate of 3.605%.

What modeling assumptions can we make about the futures price $F_t$ that would support a price like that?

#### CSO Options

Calendar spread options, or CSOs, are options on calendar spreads.
How will you price an option on an underlying that can have a negative price?

## Root-finding algorithms

Implying vol from options prices requires a choice of root finding algorithms.
In this notebook, I have relied on some off-the-shelf implementations of
Brent's method [from `scipy.optimize`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brentq.html#brentq)
and you can also see that's what [QuantLib is doing](https://github.com/lballabio/QuantLib/blob/e3bf39e73e79d5a15e1b23a314f126e35711298e/ql/instruments/impliedvolatility.cpp#L75). You can read Brent's original paper or most numerical recipes books,
although [Wikipedia](https://en.wikipedia.org/wiki/Brent%27s_method) has a pretty
good overview.

#### Keys to the algorithm
* Pick $a$ and $b$ such that $f(a) f(b) < 0$.
    * For continuous $f$, there must be a root between $a$ and $b$.
* Update $a_n$ and $b_n$ such that $f(a_n) f(b_n) < 0$.
    * Bisection method: consider $s = (a_n+b_n)/2$, update either $a_n$ or $b_n$.
    * Brent's method: Make better guesses about $s$ using secant, bisection, and inverse quadratic transforms.
* Stop when $f(b_n) = 0$ or $| a_n-b_n | < \epsilon$. See, e.g., (the scipy implementation)[https://github.com/scipy/scipy/blob/b1296b9b4393e251511fe8fdd3e58c22a1124899/scipy/optimize/Zeros/brenth.c#L85]

Apply this with $f$ your option pricing function minus your target price as a function of volatility.
#### 1. How do you pick $a$ and $b$ to start?
#### 2. How can you optimize this for the specific problem of fitting volatility to market prices?